# Assignment 6: 75 points + 5 points extra credit
## More on Ensembles, Feature Importance, KMeans clustering, Dimensionality Reduction

### IMPORTANT: 
#### You MUST read everything in tnis notebook CAREFULLY, including ALL code comments.  If you do not, then you may easily make mistakes.

Yet again we will use Yellowbrick, for Silhouette Scoring on KMeans:

Be sure to review the class slides if you need to. (But read the comments in this notebook first.)

Some relevant documentation can be found for KMeans and the Silhouette Score visualizer at these 2 URLs:

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

https://www.scikit-yb.org/en/latest/api/cluster/silhouette.html




In [ ]:
# Task 1: 5 points.  Set up environment

####################################################################################
# If some of these do not import properly, you may need to install them and re-run #
####################################################################################

import keras
import playsound
import sklearn
import tensorflow
import time
    
import matplotlib         as mpl   
import matplotlib.pyplot  as plt
import numpy              as np   
import pandas             as pd

from keras.datasets          import cifar10  
from playsound               import playsound
from pprint                  import pprint   

from sklearn.cluster         import KMeans
from sklearn.decomposition   import PCA
from sklearn.ensemble        import BaggingClassifier, ExtraTreesClassifier, RandomForestClassifier, VotingClassifier
from sklearn.linear_model    import SGDClassifier, LogisticRegression
from sklearn.metrics         import confusion_matrix, precision_recall_curve, precision_score, recall_score, f1_score, silhouette_score, homogeneity_score, completeness_score
from sklearn.model_selection import cross_val_predict, cross_val_score, GridSearchCV
from sklearn.pipeline        import make_pipeline
from sklearn.preprocessing   import StandardScaler
from sklearn.svm             import LinearSVC, SVC
from sklearn.tree            import DecisionTreeClassifier, export_graphviz

from yellowbrick.classifier  import ClassBalance, ClassificationReport, ClassPredictionError, ConfusionMatrix
from yellowbrick.cluster     import SilhouetteVisualizer

np.random.seed(42) 

%matplotlib inline 

'Done'

In [ ]:
# I modified these function definitions from Géron's ch. 9 
# notebook.  We will use them below to display  
# a Voroni plot of KMeans clusters.

def plot_data(X):
    plt.plot(X[:, 0], X[:, 1], 'k.', markersize=2)

def plot_centroids(centroids, weights=None, circle_color='w', cross_color='r'):
    if weights is not None:
        centroids = centroids[weights > weights.max() / 10]
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='o', s=5, linewidths=8,
                color=circle_color, zorder=10, alpha=0.9)
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='x', s=3, linewidths=3,
                color=cross_color, zorder=11, alpha=1)

def plot_decision_boundaries(clusterer, X, resolution=1000, show_centroids=True,
                             show_xlabels=False, show_ylabels=False):
    mins = X.min(axis=0) - 0.1
    maxs = X.max(axis=0) + 0.1
    xx, yy = np.meshgrid(np.linspace(mins[0], maxs[0], resolution),
                         np.linspace(mins[1], maxs[1], resolution))
    Z = clusterer.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(Z, extent=(mins[0], maxs[0], mins[1], maxs[1]),
                cmap="Pastel2")
    plt.contour(Z, extent=(mins[0], maxs[0], mins[1], maxs[1]),
                linewidths=1, colors='k')
    plot_data(X)
    if show_centroids:
        plot_centroids(clusterer.cluster_centers_)

    if show_xlabels:
        plt.xlabel("$x_1$", fontsize=14)
    else:
        plt.tick_params(labelbottom=False)
    if show_ylabels:
        plt.ylabel("$x_2$", fontsize=14, rotation=0)
    else:
        plt.tick_params(labelleft=False)

In [ ]:
# Here is the code to load and prep the CIFAR-10 data

np.random.seed(42) # Make this notebook's output stable across runs

(X_train, y_train), (X_test, y_test) = cifar10.load_data() 

# Normalize the data
X_train  = X_train.astype('float32')
X_test   = X_test.astype('float32')
X_train /= 255.0  # The largest number is 255, and the smallest 0
X_test  /= 255.0  # So this division will normalize the data.

X_train_flat = X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2]*X_train.shape[3])
X_test_flat  = X_test.reshape(X_test.shape[0],   X_test.shape[1]*X_test.shape[2]*X_test.shape[3])

# We also have to use ravel to change the target values (the values we want to predict). 
y_train = np.ravel(y_train)
y_test  = np.ravel(y_test)

LABEL_NAMES = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

'Done' 


#### Now let's consider a bagging model, which is very similar to a random forest model, but not identical.

The primary difference is that, for each time a tree node is split into a branch, a bagging model chooses the best feature from among ALL of the features, whereas a random forest model only chooses from a subset of all features. By default the random forest will take a random sample of the square root of the number of features, and find the best feature from that subset.   

This difference has an impact on how random forest performs vs. bagging. Think of our CIFAR-10 dataset. Bagging takes a LOT longer to run than random forest because bagging will have to test all 3072 features for each tree branch to find the best feature. But random forest only has to test 55 features – the square root of 3072! 
 
In Assignment 5 the following random forest model building 1000 trees had a test score of 0.4937 and took about 3 minutes to run (on my computer):

     RandomForestClassifier(n_jobs=-1, random_state=42, n_estimators=1000)

Yet a bagging model that builds only 100 trees takes about 20 minutes, so imagine how long it would take if we did bagging with 1000 trees!  In the next cell we will try a modification to a pure bagging model that still takes time, but should give us better accuracy than pure bagging.  It's called a 'random patches' model.

Regarding Random Patches, Géron writes that, "This technique is particularly useful when you are dealing with high-dimensional inputs (such as images)... Sampling features results in even more predictor diversity, trading a bit more bias for a lower variance."

Since we are working with images, it sounds like this algorithm is a good candidate for us to use.


In [ ]:
# Task 2: 10 points 

# Random Patches Bagging
# To modify a pure bagging model to a random patches bagging model, just add this  
# parameter to the call to BaggingClassifier: bootstrap_features=True

# Because bagging models, including random patches, are quite slow we will 
# add a timer and a call to playsound to let us know when it's finished.

# 5 points: Define a random patches model by calling BaggingClassifier with 
#           bootstrap_features, a random state of 42, and set n_jobs to use
#           all available cores, but have it build only 50 trees. Also add
#           verbose=True so you can track the training progress. Save this random
#           patches model in the the variable rndPatches
# 2 points: Fit the model on the training data.
# 1 point:  Add a line of code to print the model's score on the test data.
# 2 points: Add the code from previous assignments that you need to calculate and display
#           this model's time, and also to notify yourself when it's done with playsound.

####################  insert your code below for 10 points ####################

   

rndPatches = 










########################### Your code ends above ##############################

# On my computer this took 12 mins and had accuracy of 0.4507


#### On the same number of trees Random Patches is indeed more accurate on CIFAR-10 than random forest. 

On my laptop the score with 100 trees (instead of the 50 we just did) is 0.4695, but at the cost of 21 minutes. Of course, we are not overly excited about 0.4695 because when we built a random forest model with unlimited depth and 1000 trees, we obtained 49.37% accuracy as noted above.

However, Géron points out yet another variation to bagging that he refers to as 'Extremely Randomized Trees' that uses ExtraTreesClassifier.  This approach has a significant difference from both Random Forest and Bagging that dramatically reduces the compute time.  Let's look at it.

In [ ]:
# Task 3: 10 points     

# See Géron's ch.7 section on 'Extra Trees' where, again, 
# we see the trade off of adding more bias for reduced variance.
# Let's try it with the same settings we used for our best model
# yet, which was the 1000 tree random forest that got 0.4937 and
# took only about 3 minutes to run (again, on my computer).

# 5 points:  Use ExtraTreesClassifier to build an ensemble
#            of 1000 trees, with no maximum depth, using a
#            random state of 42 and all available CPU cores.
#            Save the model into variable xRndTrees
# 1 point:   Train xRndTrees
# 1 point:   Evaluate xRndTrees on the test data and print
#            that score with an appropriate explanatory message
# 3 points:  Add the timing code and playsound.

####################  insert your code below for 10 points ####################



xRndTrees = 

 






########################### Your code ends above ##############################

# 0.4965, indeed better than the rf model we did earlier with exactly the same parameter settings
# and at just over 2 mins, is not only MUCH faster than Bagging but also about 1/3 FASTER than the rf 
# This is now our best model so far.


#### Exploring Feature Importance
With the ExtraTreesClassifier (and also with RandomForestClassifier, but not bagging models) you can get some very interesting information about the features of your trained model. There is a parameter you can inspect after training that indicates the relative importance of each feature in your training data.  In Fig. 7-6 of Géron, there is a visualization (left) of the the feature importance for a digit.  The image on the right is a similar one I created for the CIFAR-10 images:

<img src='mnist_feature_importance_plot.png' width="400" height="400"> <img src='cifar10_feature_importance_plot.png' width="420" height="420">

Not only are the bright pixels the ones that often carry the most useful information, but it's also interesting to notice how many completely or nearly black pixels there are.  Those indicate relatively useless features.  I will return to this point shortly.  But first, let's see how we created the image for CIFAR-10 on the right.

In [ ]:
# Task 4: 10 points  

# In both images above there is a 2-dimensional matrix where each value
# represents the importance of the pixel that corresponds to that row, column.
# The code below is for the image on the right, but we are using color images, 
# not black & white which was used for the image on the left, and each 
# RGB channel has its own feature importance value.

# Read what Géron writes about feature_importances_ as he discusses Figure 7-6 
# and you may realize that what WE need is not a shape of (32, 32, 3) but a shape 
# of (32, 32) whose values are the SUM of the 3 real numbers in the 3rd dimension.

# I have written most of the code for you. Your code will be inserted into
# the body of a LOCAL function called 'addRGB' that is enclosed by a function
# called 'plotPixelImportance'.  If you have never defined such a local 
# function before, it just means that you can ONLY call that local function
# from inside the enclosing function.  If you try to call it outside
# of the enclosing function, you'll get an error.

# Here are the points for your code:
# 3 points:  Initialize a numpy matrix in the shape of 32 by 32 since our images
#            have that many pixels (32 * 32 = 1024 pixels). Set the initial
#            values of this matrix to all zeros. (Use Google if you don't
#            know how to create a numpy matrix initialized to zeros.) Save the
#            matrix into the variable 'newImg'
# 5 points:  write Python code to replace each 0 in newImg with the sum of
#            the 3 numbers in the third dimension of 'img', which is the 
#            argument to the local function addRGB. Think about  
#            the shape of the img argument, and how you can access
#            its third dimension. (Google is your friend) It may help
#            to create a temporary notebook cell, manually build a small
#            matrix of shape (2, 3, 3) with random numbers, and test out
#            some code, then recycle that code here when its working.
#            Hint: one solution is to use an outer loop that iterates
#            over matrix rows and an inner loop iterating over columns.
#  2 points: Return the modified newImg matrix as the value of the local 
#            function addRGB

# It may also help you to READ the code below, especially the reshape code 
# beneath the local function before you begin.

def plotPixelImportance(data):
    # This is Géron's function for Fig. 7-6, modified for the CIFAR10 dataset.
    # Plot a heatmap of the importance for classification of the 1024 pixels,
    # where each pixel has 3 color channels 
    
    # 'data' is a color image with shape (32, 32, 3)
    def addRGB(img):
        '''Convert the 'data' shape(32, 32, 3) to the newImg shape(32, 32) by adding 
           the numbers in the 3rd dimension of 'data' and putting their sum into 
           each row/column pixel of newImg'''
        
######################### Add your code below, about 5 lines or so ##########################
        
        newImg =  
        
        
        
        
        
        
        
        
    
#################################### Your code ends above ###################################
    
    image    = data.reshape(32, 32, 3) # restore to original shape of images
    imageNew = addRGB(image)           # convert to shape(32, 32) by adding 
                                       # the feature importance values in the 3rd dimension
    plt.imshow(imageNew, cmap = mpl.cm.hot,
               interpolation="nearest")
    plt.axis("off")
    return(imageNew)

heatMap = plotPixelImportance(xRndTrees.feature_importances_)
cbar    = plt.colorbar(ticks=[xRndTrees.feature_importances_.min(), 
                              xRndTrees.feature_importances_.max()])

plt.show()


In [ ]:
# Task 5: 10 points     
# Chapter 9 is about unsupervised methods to explore data.
# One of the most common algorithms is K-Means, which is used
# to find naturally occurring clusters of data.  For example,
# since we are analyzing 10 categories of different pictures,
# it would be reasonable to ask if each of those 10 categories 
# somehow forms a natural cluster.  KMeans can help expore that
# hypothesis. 

# KMeans is a very fast algorithm and it groups data instances
# by how similar they are, where similarity is generally based
# on Euclidean distance. (See Géron and my lecture for details.)

# So let's create a KMeans clustering of the CIFAR-10 data.
# We will use only 5000 images, which should include about 
# 500 of each category.

# KMeans is UNsupervised, and does not know in 
# advance how many (or even if) natural clusters exist. Hence,
# there is a hyperparameter that controls how many clusters
# are created. It is common to tune this n_clusters hyperparameter
# and analyze the results to determine the optimal number
# of clusters to create.  But since we already know there are
# 10 categories in CIFAR-10, we will simply set n_clusters=10
# and let it run.  Let's see if it shows anything interesting.

# 3 points: Use the preset variable numImages to extract
#           that number of images from X_train_flat and save them
#           into the variable cifarImgs
# 2 points: Save the corresponding labels for those images
#           into the variable cifarLabels (we will use that variable
#           below in another cell, not this cell) Be careful here!
#           Do not simply get the value of LABEL_NAMES.  Those are
#           string labels like 'cat'.  The true labels, which are
#           integers, are the values of y_train.
#           You want the labels that match the cifarImgs that you 
#           just did.  So you can actually get those labels in
#           a similar manner by using y_train
# 3 points: Create a KMeans model and use the preset numClusters
#           to tell KMeans how many clusters to create with 
#           a random state of 16 (NOT 42 or any other number!)
#           Use the URL at the beginning of this notebook to 
#           find the parameter name to set with numClusters
#           Save your model into the variable km         
# 2 points: Call the km model's fit method on cifarImgs

######################### Add your code below ##########################

numImages   = 5000 # you can try different values if you like
numClusters = 10   # do not change this parameter for the assignment
cifarImgs   =                           # get 1st numImages train images
cifarLabels =                           # and their labels. This
                                        # var will be used later in
                                        # the notebook
km          = 



######################## Your code ends above ##########################

# In the next cell we will inspect this model to get
# insight into how good it is
'Done'

In [ ]:
# Task 6: 10 points   

# https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html
# The Silhouette Score (described in Géron) is a good way 
# to evaluate how good a KMeans cluster is.

# from sklearn.metrics import silhouette_score

# In the text and lecture, you will remember that a Silhouette score 
# close to 1 means that the clusters are well separated, which is good. 
# If close to -1 they're NOT, and close to 0 means there is a lot
# of ambiguity in the cluster assignments of data observations.

# We will use the yellowbrick implementation for Silhouette scores
# because it is so easy to use. In contrast, go look at Géron's code
# in his chapter 9 notebook and notice how much work it would be to
# modify it to do what we want here.

# 2 points: Define the same KMeans model as in the previous cell and save it in kmYB
# 3 points: Call yellowbrick's SilhouetteVisualizer and pass it kmYB, 
#           colors='yellowbrick' and size=(960, 640)
#           Save the visualizer in ybViz
# 3 points: Call the fit method of ybViz on cifarImgs, similar to km.fit in the previous cell
# 2 points: Call the show method of ybViz

######################### Add your code below ##########################

print('\nThe Silhouette Score is:', silhouette_score(cifarImgs, km.labels_))  
# labels_ tells us which cluster each observation was assigned to

kmYB  =                                                 # Create same KMeans model because training
                                                        # needs to happen in scope of the visualizer
ybViz = 

                                                       
    
    
    

######################## Your code ends above ##########################


#### Oops, KMeans doesn't really do so well on the CIFAR-10 data, does it?  

The Silhouette Score so close to zero tells us that it's bad.  The Silhouette visualization confirms it, and also provides a bit of explanation because notice how many of the clusters have a significant portion of their observations with negative-valued silhouette coefficients.  While KMeans does not work well on this particular dataset (it does for many others), we will build one more KMeans model so we can see something else that is interesting. But first, let's do a short extra credit task for 5 points.

In [ ]:
# Task 7: 5 EXTRA CREDIT points   

# Before we decide to move on from KMeans, let's do a bit more.  
# know that it doesn't work well on the raw CIFAR-10 data, but aren't
# you curious to know more?

# First of all, look how easy it is to extract the 'inertia' from
# a KMeans model:
print('Inertia:\t', km.inertia_) 

# You'll remember that this metric measures the average squared distance
# between each data point and its closest centroid.  Since tight
# clusters are desired, this means that an inertia close to 0 is 
# also desired.  When you run this cell you'll see the inertia.
# But we won't know if the number displayed is good or bad
# since we have nothing to compare it to.  We did not try other
# values of k.  But there are other metrics that may be useful
# to us.

# If you know what the ground truth labels are for your data (and we do, 
# even though we did not use that info for the last two tasks),
# then here are two more metrics that can give a bit more insight:
#    homogeneity:  each cluster contains only members of a single class.
#    completeness: all members of a given class are assigned to the same cluster.
# Both of these range from 0 to 1 with 1 being the best possible.

# These are not discussed by Géron, but the sklearn documentation for homogeneity is at:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.homogeneity_score.html?highlight=homogeneity_score
# and completeness is here:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.completeness_score.html#sklearn.metrics.completeness_score

# We can also extract these metrics as long as we provide the 
# ground truth labels, which (if you did it correctly in Task 5) are already
# in the variable 'cifarLabels'

# KMeans has a 'predict' method, even though it is not a supervised
# learning algorithm.  It will simply predict the most appropriate cluster
# assignment for its argument based on the trained KMearns model. We can
# use it here to predict the clusters on the cifarImgs that were used
# to created the KMeans model itself.

# 3 points: Use the 'km' model from Task 5 to predict the class assignments
#           for the cifarImgs.  Save those predictions into predictedLabels
# 1 point:  Call the homogeneity_score function, passing it the two 
#           arguments cifarLabels that you set in Task 5, and
#           predictedLabels that you just set moments ago.  Put that code
#           as the 2nd argument to the first print statement below.
# 1 point:  Call the completeness_score function, passing it the same two
#           arguments that you used for homogeneity.  Put that code
#           as the 2nd argument to the second print statement below. 
######################### Add your code below ##########################

predictedLabels = 

print('Homogeneity:\t',  )
print('Completeness:\t', )

######################### Add your code above  ##########################

# Assuming you also see these latter two metrics near 0.09 it would
# appear that KMeans may be hopeless. But let's try something else 
# before we stop using KMeans.

In [ ]:
# Task 8: 10 points   

# As you know, we skipped chapter 8, but I want to show you
# what is possible with its techniques so you may be
# interested enough to read the whole chapter.
# With our CIFAR-10 data each of the 32 by 32 pixels
# has 3 colors, so when we flatten that shape (32, 32, 3) 
# matrix we end up with 3072 numbers in a vector.  Each number
# will be treated as a feature by any ML algorithm.  That is not
# only a lot of features, but we saw in the feature importance
# color map above that many of those features are not
# actually very important.  Is there some way we can convert
# those 3072 features into a smaller number of features
# such that the new ones ARE ALL important?  

# Yes there is, and we will use dimension reduction from Chapter 8 
# to do so. The most common dimension reduction algorithm is called 
# Principal Component Analysis, or PCA for short.  We were unable
# to visualize the clusters of our KMeans algorithm above 
# because there is no way to plot 3072 features on a 2-dimensional
# computer screen.  But using PCA we can reduce them down to only 2
# features, which we CAN plot.  In fact, it's easy to do.

# from sklearn.decomposition import PCA

# I have written some of the code below for you,
# but there are still some lines to complete.  
# For 10 points total:
#   3 points: Call PCA and pass it 2 arguments: 
#        the first is the number of principal components 
#        that we want, which is 2, so we can plot it.  
#        So set n_components to 2 as your first argument.  
#        Also set random_state to 2 and use that as the 2nd argument.
#        Save the value of that PCA definition into pcaData
#   2 points: Now call the fit_transform method of PCA with cifarImgs, 
#        which will perform the actual PCA transformation of the 3072
#        features down to only 2! (the most important 2, by the way)
#        Save this value back into the variable pcaData
#   2 points: I have defined the same KMeans model as we used earlier
#        and saved it in variable kmPCA.  So now call its fit method
#        with pcaData instead of the original cifarImgs data
#   3 points: Call the plotting function defined near the top of this
#        notebook, passing it both kmPCA and pcaData as arguments.
#        This plotting function is called plot_decision_boundaries


######################### Add your code below ##########################

pcaData = 
pcaData = 

print('\n Confirm we have 5000 rows but only 2 columns now:', pcaData.shape)

pcaData = pcaData.astype('double')                        # If this is omitted, you
                                                          # will see a data type error
kmPCA   = KMeans(n_clusters=numClusters, random_state=16) # Same model we used above


# Put your code to fit kmPCA on pcaData AFTER this comment:
  

# Let's now look at the Voroni visualization using the functions
# defined near the top of this notebook.
# The red dots inside a white circle show the location of the cluster centers

# Put your call to plot_decision_boundaries AFTER this comment, but
# BEFORE the call to plt.show()



plt.show()  # This will show the Voroni visualization with boundaries

######################## Your code ends above ##########################


In this Voroni visualization the red dots inside white circles are the locations of the cluster centers. If you are wondering why I had you use random_state = 16 for the KMeans model and random_state = 2 for the PCA data, it's because of some weirdness in this Voroni visualization.  With other random states it doesn't draw all of the boundaries properly.  I don't know why.  It's Géron's code and I didn't study it that carefully.

The point of this task is to show you that you can use PCA dimensionality reduction to shrink large dimensional data down to 2 dimensions, which then allows you to visualize it.  Pretty cool, huh?  But notice another thing:  The plotted points don't really appear visually as 10 distinct clusters.  If your ignore the class boundary lines, you can see it simply looks like one big cluster.  That is why our Silhouette Score was so bad and why the Silhouette visualization earlier also looked bad. 

I think one reason for this is due to the large dimensionality, as well as the diversity of images even of the same object.  Not all cats or boats look the same as others, and many images are from different angles or viewpoints, hence the location of the image in high-dimensional space varies quite a lot, even for the same category.  This explanation also makes sense given that we still haven't found a classifier that gives us even 50% accuracy.  



In [ ]:
# Task 9: 10 points 

# Let's see what else we can do with PCA. We'll use what you just 
# learned about building a pca-reduced model, but we'll do more 
# than 2 features. This time, however, instead of calling 
# the fit_transform method as we did above, we will fit the data 
# and then transform it separately.  I'll explain why below.

# 2 points: Build a PCA model that creates 40 PCA components with a
#           random state of 42. (Yes, we are returning to 42 now.)
#           Save it into pca40
# 2 points: Call the fit  method on pca40, but this time
#           call it on ALL of X_train_flat. Do not save the results
#           into any variable.  
# 1 point:  Call the transform method of the fitted pca40 model,
#           passing it X_train_flat, but now save THIS result
#           into the new variable X_train_PCA
# 1 point:  Also transform the TEST data X_test_flat by 
#           calling the transform method of pca40 a second time,
#           saving the result into X_test_PCA

# This is why we separated the fit from the transform this time.
# It is CRUCIALLY IMPORTANT to apply any data transformations on
# your test data in an IDENTICAL way to your transformations of your 
# training data to avoid data snooping.

# After you have added your code for these parts, there are 2 print 
# statements. If you do not see 40 columns in both X_train_PCA and 
# X_test_PCA then you have done something wrong in the previous steps.

# Continue with more code after the print statements by building
# a new ExtraTreesClassifier in the following way:

# 2 points: Copy the ExtraTreesClassifier definition from Task 3
#           and save it into the variable pcaXTrees
# 2 points: Train pcaXTrees on X_train_PCA and y_train 

# When you run this cell, there are 2 more lines of code at the
# bottom that will evaluate your new model and print the test
# data score.  If you have done this correctly, you should see
# a new accuracy that is better than 50% for the first time!

######################### Add your code below ##########################

pca40 = 



X_train_PCA =                               # Apply PCA reduction to both train 
X_test_PCA  =                               # and test data

print(X_train_PCA.shape)
print(X_test_PCA.shape)

pcaXTrees   = 

 

    


######################## Your code ends above ##########################

testAcc = pcaXTrees.score(X_test_PCA, y_test)
print('\nTest accuracy of extremely randomized trees with 40 principal components:', testAcc)


Assuming you got the expected results, your model should be a bit higher than 51% test accuracy! This is because those first 40 principal components account for about 80% of the variance in the data (I determined that by doing some separate analysis that I did not include here.)  What that means is that we have taken 3072 features, many of which carried little useful information, and reduced them down to 40 features which carry excellent information. That's why we got better results.  This will not happen every time for every dataset where you use PCA, but when you have high dimensional data and you are getting disappointing results, try PCA and you may very well see an improvement.  We are almost done.  

Run the next cell where we take the first THREE of the 40 principal components from our pcaXTrees model and use that 3rd component as a third dimension to display a 3-D visualization of the 10000 test data points.

In [ ]:
points_to_plot = 10000

fig = plt.figure(1, figsize=(8, 6))
ax  = fig.add_subplot(111, projection="3d", elev=30, azim=134) 

ax.set_position([0, 0, 0.95, 1])

ax.scatter(X_test_PCA[0:points_to_plot, 0], 
           X_test_PCA[0:points_to_plot, 1], 
           X_test_PCA[0:points_to_plot, 2], 
           c         = y_test[0:points_to_plot], 
           cmap      = plt.cm.nipy_spectral, 
           edgecolor ="k")

plt.title('First 3 Principal Components \nTest accuracy = {:.4f}'.format(testAcc))
plt.show()


### I hope you are now encouraged to read chapter 8 after the class is over.  Keep learning!